# PDF Embeddings Generation

This notebook reads a PDF file, extracts text, and generates embeddings using the embedding model configured in the environment variables.

**Configuration:**
- If `USE_OLLAMA=true`: Uses Ollama with `nomic-embed-text:latest`
- Otherwise: Uses Google Generative AI with `models/embedding-001` (requires `GOOGLE_API_KEY`)

In [27]:
# Install required packages (run once if needed)
# %pip install langchain-community langchain-ollama langchain-google-genai pypdf python-dotenv

In [28]:
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_vertexai import VertexAIEmbeddings

use_ollama = os.getenv("USE_OLLAMA", "false").lower() == "true"
print(f"📊 Configuration: USE_OLLAMA = {use_ollama}")

📊 Configuration: USE_OLLAMA = False


In [29]:
# Initialize the embedding model based on environment configuration
if use_ollama:
    ollama_base_url = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
    embeddings = OllamaEmbeddings(
        model="nomic-embed-text:latest",
        base_url=ollama_base_url
    )
    print(f"✅ Initialized Ollama embeddings: nomic-embed-text:latest")
    print(f"   Base URL: {ollama_base_url}")
else:
    project_id = os.getenv("GOOGLE_CLOUD_PROJECT_ID")
    location = os.getenv("GOOGLE_CLOUD_LOCATION", "asia-south1")
    
    if not project_id:
        raise ValueError("GOOGLE_CLOUD_PROJECT_ID environment variable is required when not using Ollama")

    embeddings = VertexAIEmbeddings(
        model="gemini-embedding-001",          # or "models/text-embedding-004" etc.
        project=project_id,                # ← Required for Vertex AI
        location=location,                       # Common location; check supported regions
        dimensions=768
    )
    print(f"✅ Initialized Vertex AI embeddings: gemini-embedding-001")
    print(f"   Project: {project_id}")
    print(f"   Location: {location}")

✅ Initialized Vertex AI embeddings: gemini-embedding-001
   Project: notebooklm-clone-483513
   Location: asia-south1


/var/folders/wb/8pdc7h9s4md3llv724zwpzdw0000gn/T/ipykernel_56037/1490009498.py:17: DeprecationWarning: Use [`GoogleGenerativeAIEmbeddings`][langchain_google_genai.GoogleGenerativeAIEmbeddings] instead.
  embeddings = VertexAIEmbeddings(


In [30]:
# Check Google Application Credentials being used by Google Cloud SDK
from google.auth import default

print("🔐 Google Cloud SDK Credentials Check:")
# print(f"   GOOGLE_APPLICATION_CREDENTIALS env var: {os.getenv('GOOGLE_APPLICATION_CREDENTIALS', 'Not set')}")

try:
    credentials, project = default()
    print(f"   ✅ Credentials loaded successfully")
    print(f"   Project ID: {project}")
    
    if hasattr(credentials, 'service_account_email'):
        print(f"   Service Account Email: {credentials.service_account_email}")
    elif hasattr(credentials, 'client_email'):
        print(f"   Service Account Email: {credentials.client_email}")
    
except Exception as e:
    print(f"   ❌ Error loading credentials: {e}")

🔐 Google Cloud SDK Credentials Check:
   ✅ Credentials loaded successfully
   Project ID: notebooklm-clone-483513
   Service Account Email: 743023577634-compute@developer.gserviceaccount.com


In [32]:
# Load the PDF file
pdf_path = Path("backend/src/upload_graph/Sample-Accounting-Income-Statement.pdf")

# Load PDF using LangChain's PyPDFLoader
loader = PyPDFLoader(str(pdf_path))
docs = loader.load()
all_texts = [doc.page_content for doc in docs]

embeddings.embed_documents(all_texts)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: gemini-embedding. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.', 'status': 'RESOURCE_EXHAUSTED'}}